# This python notebook is used to perform Exploratory Data Analysis (EDA) on the Lot table from TED data.


Select only the Lot table from the postgres database

In [1]:
from ted_data_eu.services.etl_pipelines.postgres_etl_pipeline import POSTGRES_URL, SQLALCHEMY_ISOLATION_LEVEL
import sqlalchemy
import pandas as pd

CSV_FILE_PATH = '/home/mihai/work/meaningfy/ted-data-eu/notebooks/exported_data.csv'
DB_TABLE = "Lot"
DB_QUERY = f"""
            SELECT * FROM public."{DB_TABLE}"
            """


Connect to the database and download the information in a csv file

In [2]:
sql_engine = sqlalchemy.create_engine(POSTGRES_URL, echo=False, isolation_level=SQLALCHEMY_ISOLATION_LEVEL)
with sql_engine.connect() as sql_connection:
    df = pd.read_sql(DB_QUERY, sql_connection)
df.to_csv (CSV_FILE_PATH, index = False)

Load the file and view the basic information

In [ ]:
data_table = pd.read_csv(CSV_FILE_PATH)
print(data_table.info())

For each field in the table, represent the distribution by the number of characters (bar chart)
and calculate std, average, percentile1, percentile99, min, max, median, iqr, z-score.

In [ ]:
import plotly.graph_objects as go

deleted_columns = ['LotId', 'ProcedureId', 'isUsingEUFunds']
adjusted_df = data_table.drop(columns=deleted_columns)


def add_sphere_trace(fig, x, color, label):
    """
    Creates spheres of indicators in the histogram
    :param fig: the figure itself
    :param x: calculated indicators
    :param color: color
    :param label: the name of the sphere representing the indicator
    :return: the ID of the added view
    """
    fig.add_trace(go.Scatter(
        x=[x],
        y=[0],
        mode='markers',
        marker=dict(
            size=12,
            symbol='circle',
            color=color,
            line=dict(color='black', width=1),
            opacity=0.7
        ),
        name=label
    ))


def generate_histogram(data, column, nbinsx):
    """
    Generates the histogram of each field from the table
    :param data: the length of each field
    :param column: name of the column
    :param nbinsx: number of bins
    :return: the figure with the histogram result
    """

    fig = go.Figure()

    std = data.std()
    average = data.mean()
    percentile1 = data.quantile(0.01)
    percentile99 = data.quantile(0.99)
    min = data.min()
    max = data.max()
    median = data.median()
    z_score = (data - average) / std
    iqr = data.quantile(0.75) - data.quantile(0.25)

    fig.add_trace(go.Histogram(x=data, nbinsx=nbinsx))
    std_line_color = 'white'
    std_line_width = 6

    fig.add_shape(
        type="line",
        x0=0,
        y0=std,
        x1=nbinsx,
        y1=std,
        line=dict(color=std_line_color, width=std_line_width, dash="dash"),
        layer='below'
    )

    add_sphere_trace(fig, average, 'red', 'Average')
    add_sphere_trace(fig, std, 'white', 'STD')
    add_sphere_trace(fig, percentile1, 'blue', 'Percentile 1')
    add_sphere_trace(fig, percentile99, 'blue', 'Percentile 99')
    add_sphere_trace(fig, min, 'green', 'Min')
    add_sphere_trace(fig, max, 'green', 'Max')
    add_sphere_trace(fig, median, 'orange', 'Median')
    add_sphere_trace(fig, z_score, 'purple', 'Z-Score')
    add_sphere_trace(fig, iqr, 'yellow', 'IQR')

    title = f'<b>Distribution of the length of the string for {column}</b><br>'
    title += f'Std: {std:.2f}, Average: {average:.2f}, Percentile 1: {percentile1:.2f}, Percentile 99: {percentile99:.2f}<br>'
    title += f'Min: {min:.2f}, Max: {max:.2f}, Median: {median:.2f}, IQR: {iqr:.2f}, Z_Score: {z_score.values} <br>'

    fig.update_layout(
        title=title,
        xaxis_title='The length of the string',
        yaxis_title='Number of records',
        title_font=dict(size=14)
    )
    fig.show()


for column in adjusted_df.columns:
    if adjusted_df[column].dtype == 'object':
        adjusted_df['number_of_characters'] = adjusted_df[column].str.len()
        generate_histogram(adjusted_df['number_of_characters'], column, nbinsx=100)
    elif adjusted_df[column].dtype == 'int64':
        generate_histogram(adjusted_df[column], column, nbinsx=100)


Calculate KPIs (data completeness, consistency, uniqueness)

In [ ]:
def calculate_data_consistency(df, column):
    """
    Calculates the data consistency for a specific column in the DataFrame
    :param df: dataFrame containing the data
    :param column: name of the column for which data consistency is calculated
    :return:  data consistency percentage as a floating-point number
    """
    number_of_records = len(df[column])
    adjusted_df[f'length {column}'] = adjusted_df[column].str.len()
    percentile_1 = adjusted_df[f'length {column}'].quantile(0.01)
    percentile_99 = adjusted_df[f'length {column}'].quantile(0.99)
    adjusted_df[f'threshold {column}'] = adjusted_df[f'length {column}'].between(percentile_1, percentile_99)
    number_of_items = adjusted_df[f'threshold {column}'].sum()
    data_consistency = (number_of_items / number_of_records) * 100
    return data_consistency

def calculate_data_completeness(df, column):
    """
    Calculates the data completeness for a specific column in the DataFrame
    :param df: dataFrame containing the data
    :param column: name of the column for which data completeness is calculated
    :return: data completeness percentage as a floating-point number
    """
    number_of_records = len(df[column])
    number_of_records_not_null = df[column].notnull().sum()
    data_completeness = (number_of_records_not_null / number_of_records) * 100
    return data_completeness

def calculate_data_uniqueness(df, column):
    """
    Calculates the data uniqueness for a specific column in the DataFrame
    :param df: dataFrame containing the data
    :param column: name of the column for which data uniqueness is calculated
    :return: data uniqueness percentage as a floating-point number
    """
    number_of_records = len(df[column])
    number_of_unique_values = df[column].nunique()
    data_uniqueness = (number_of_unique_values / number_of_records) * 100
    return data_uniqueness

columns_to_calculate = ["LotTitle", "LotEstimatedValueEUR", "LotDescription"]
for column in columns_to_calculate:
    if column == "LotEstimatedValueEUR":
        total_values = len(adjusted_df['LotEstimatedValueEUR'])
        num_zeros = (adjusted_df['LotEstimatedValueEUR'] == 0).sum()
        num_non_zeros = (adjusted_df['LotEstimatedValueEUR'] != 0).sum()
        percentage_zeros = (num_zeros / total_values) * 100
        percentage_non_zeros = (num_non_zeros / total_values) * 100
        print(f"The KPI data completeness in the LotEstimatedValueEUR field is {percentage_non_zeros:.2f}% and the percentage with zero values is {percentage_zeros:.2f}%.")

    elif column == "LotDescription":
        data_completeness = calculate_data_completeness(adjusted_df, column)
        data_consistency = calculate_data_consistency(adjusted_df, column)
        print(f"The KPI data completeness in the {column} field is {data_completeness:.2f}%.")
        print(f"The KPI data consistency in the {column} field is {data_consistency:.2f}%.")

    else:
        data_consistency = calculate_data_consistency(adjusted_df, column)
        data_completeness = calculate_data_completeness(adjusted_df, column)
        data_uniqueness = calculate_data_uniqueness(adjusted_df, column)
        print(f"The KPI data consistency in the {column} field is {data_consistency:.2f}%.")
        print(f"The KPI data completeness in the {column} field is {data_completeness:.2f}%.")
        print(f"The KPI data uniqueness in the {column} field is {data_uniqueness:.2f}%.")